In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install -U -q "google-genai==1.7.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 3.7 MB/s eta 0:00:00


In [3]:
from google import genai
from google.genai import types

genai.__version__

'1.7.0'

In [4]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
GOOGLE_API_KEY = user_secrets.get_secret("GOOGLE_API_KEY")

In [5]:
client = genai.Client(api_key=GOOGLE_API_KEY)

In [6]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [7]:
import requests
from datetime import datetime

def callAPI(city): 
    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid=426b3d62a297848efcc74b59394f6f77"
    response = requests.get(url).json()

    if response.get("cod") != 200:
        print("❌ Error:", response.get("message", "Unknown error"))
        return None

    name = response.get("name", city)
    country = response["sys"].get("country", "")
    weather_desc = response["weather"][0].get("description", "").title()
    
    lat = response["coord"]["lat"]
    lon = response["coord"]["lon"]

    temp_k = response["main"]["temp"]
    feels_like_k = response["main"]["feels_like"]
    temp_c = round(temp_k - 273.15, 2)
    feels_like_c = round(feels_like_k - 273.15, 2)
    humidity = response["main"]["humidity"]
    wind_speed = response["wind"]["speed"]

    sunrise = datetime.fromtimestamp(response["sys"]["sunrise"]).strftime('%H:%M:%S')
    sunset = datetime.fromtimestamp(response["sys"]["sunset"]).strftime('%H:%M:%S')

    print(f"📍 Location: {name}, {country} ({lat}, {lon})")
    print(f"🌤️ Weather: {weather_desc}")
    print(f"🌡️ Temp: {temp_c}°C (Feels like {feels_like_c}°C)")
    print(f"💧 Humidity: {humidity}%")
    print(f"🌬️ Wind: {wind_speed} m/s")
    print(f"🌅 Sunrise: {sunrise} | 🌇 Sunset: {sunset}")

    return response

In [10]:
import requests
from datetime import datetime

API_KEY = "426b3d62a297848efcc74b59394f6f77"
temp_map = {}
def predictTemp(city): 
    url = f"https://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_KEY}&units=metric"
    try:
        response = requests.get(url, timeout=10)
        print("Status Code:", response.status_code)
        details = response.json().get("list")
        for date in details: 
            timestamp = date["dt"]
            converted_time = datetime.utcfromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')
            temp_map[converted_time] = date["main"]["temp"]
        print(temp_map)
        
    except requests.exceptions.RequestException as e:
        print("Request failed:", e)

In [11]:
city = input("Enter name of city: ")
res = callAPI(city)
futurePred = predictTemp(city)
documents = [
    f"Weather in {city}: {res['weather'][0]['main']}, Temperature: {round(res['main']['temp'] - 273, 2)}°C, Feels like: {round(res['main']['feels_like'] - 273, 2)}°C, Latitude: {res['coord']['lat']}, Longitude: {res['coord']['lon']}, Upcoming days: {temp_map}"
]

Enter name of city:  Chennai


📍 Location: Chennai, IN (13.0878, 80.2785)
🌤️ Weather: Few Clouds
🌡️ Temp: 32.57°C (Feels like 39.57°C)
💧 Humidity: 75%
🌬️ Wind: 4.12 m/s
🌅 Sunrise: 00:23:17 | 🌇 Sunset: 12:51:57
Status Code: 200
{'2025-04-20 15:00:00': 31.49, '2025-04-20 18:00:00': 29.99, '2025-04-20 21:00:00': 28.14, '2025-04-21 00:00:00': 28.03, '2025-04-21 03:00:00': 30.65, '2025-04-21 06:00:00': 32.35, '2025-04-21 09:00:00': 32.42, '2025-04-21 12:00:00': 30.39, '2025-04-21 15:00:00': 28.96, '2025-04-21 18:00:00': 28.5, '2025-04-21 21:00:00': 27.86, '2025-04-22 00:00:00': 27.13, '2025-04-22 03:00:00': 29.91, '2025-04-22 06:00:00': 32.07, '2025-04-22 09:00:00': 32.06, '2025-04-22 12:00:00': 30.38, '2025-04-22 15:00:00': 28.81, '2025-04-22 18:00:00': 28.33, '2025-04-22 21:00:00': 27.77, '2025-04-23 00:00:00': 27.44, '2025-04-23 03:00:00': 30.45, '2025-04-23 06:00:00': 33.11, '2025-04-23 09:00:00': 32.33, '2025-04-23 12:00:00': 30.5, '2025-04-23 15:00:00': 29.17, '2025-04-23 18:00:00': 28.88, '2025-04-23 21:00:00': 28

In [15]:
pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 70.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 56.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 49.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 70.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.4/188.4 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [16]:
for m in client.models.list(): 
    if "embedContent" in m.supported_actions:
        print(m.name)

models/embedding-001
models/text-embedding-004
models/gemini-embedding-exp-03-07
models/gemini-embedding-exp


In [17]:
from chromadb import Documents,EmbeddingFunction,Embeddings
from google.api_core import retry
from google.genai import types
is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

In [18]:
class GeminiEmbeddingFunction(EmbeddingFunction): 
    def __init__(self, document_mode=True):
        self.document_mode = document_mode

    @retry.Retry(predicate=is_retriable)
    def __call__(self, input: Documents) -> Embeddings: 
        embedding_task = "retrieval_document" if self.document_mode else "retrieval_query"

        response = client.models.embed_content(
            model="models/text-embedding-004",
            contents=input,
            config=types.EmbedContentConfig(
                task_type=embedding_task
            ),
        )
        return [e.values for e in response.embeddings]


In [19]:
import chromadb
DB_NAME="weatherdb"

embed_fn=GeminiEmbeddingFunction()
embed_fn.document_mode=True

chroma_client=chromadb.Client()


In [20]:
db = chroma_client.get_or_create_collection(name=DB_NAME, embedding_function=embed_fn)
db.add(documents=documents, ids=[str(i) for i in range(len(documents))])

In [21]:
db.count()

1

In [26]:
from IPython.display import Markdown

embed_fn.document_mode=False
query="What is the temperature? And how will it be in Upcoming days?"

result=db.query(query_texts=[query],n_results=1)
[all_passages]=result["documents"]
# Markdown(all_passages[0])

In [ ]:
query_oneline = query.replace("\n", " ")

prompt = f"""
You are a helpful and friendly assistant that answers questions based on the reference passage below. 
When answering, please keep in mind that the person you're talking to may not be familiar with technical terms, 
so break things down in a simple, conversational way. Feel free to provide context and background to make sure the 
answer is thorough and clear. If the passage doesn't help with answering the question, you can leave it out.

QUESTION: {query_oneline}
"""


for passage in all_passages: 
    passage_oneline=passage.replace("\n"," ")
    prompt += f"PASSAGE: {passage_oneline}\n"

In [24]:
answer = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=prompt)

Markdown(answer.text)

Okay, here's the weather information for Chennai!

Right now, the temperature is 32.72°C, but it feels like 39.72°C.

For the upcoming days, here's a breakdown of the predicted temperatures:

*   **April 20, 2025:** Temperatures will range from 28.14°C to 31.49°C.
*   **April 21, 2025:** Temperatures will range from 27.86°C to 32.42°C.
*   **April 22, 2025:** Temperatures will range from 27.77°C to 32.07°C.
*   **April 23, 2025:** Temperatures will range from 28.32°C to 33.11°C.
*   **April 24, 2025:** Temperatures will range from 28.58°C to 33.66°C.
*   **April 25, 2025:** Temperatures will range from 28.49°C to 33.27°C.

So, it looks like it will be consistently warm over the next few days, with temperatures generally in the high 20s to low 30s.